# IV. Create an Interactive Geographic
Ref: 
- https://towardsdatascience.com/how-to-create-an-interactive-geographic-map-using-python-and-bokeh-12981ca0b567

If you are looking for a powerful way to visualize geographic data then you should learn to use interactive Choropleth maps. A Choropleth map represents statistical data through various shading patterns or symbols on predetermined geographic areas such as countries, states or counties. Static Choropleth maps are useful for showing one view of data, but an interactive Choropleth map is much more powerful and allows the user to select the data they prefer to view.

The interactive chart below provides details on San Francisco single family homes sales. The chart breaks down the single family home sales by Median Sales Price, Minimum Income Required, Average Sales Price, Average Sales Price Per Square Foot, Average Square Footage and Number of Sales all by neighborhood and year (10 years of data).

In [1]:
import os
import pandas as pd
from IPython.display import display
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap
plt.style.use(style='ggplot')
plt.rcParams['figure.figsize'] = (10, 6)
from matplotlib.colors import LogNorm
from scipy.stats import skew

# import necessary packages to work with spatial data in Python



pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Using Python and Bokeh
After exploring several different approaches, I found the combination of Python and Bokeh to be the most straightforward and well-documented method for creating interactive maps.

Let’s start with the installs and imports you will need for the graphs. Pandas, numpy and math are standard Python libraries used to clean and wrangle the data. The geopandas, json and bokeh imports are libraries needed for the mapping.

I work in Colab and needed to install fiona and geopandas.

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import math

import fiona
import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column



## 4.1 Load and Clean the Data

Here we are importing the data from the csv file

In [3]:
neighborhood_data = pd.read_csv('data_seloger_EDAforSpatial_part3.csv')
neighborhood_data.head()

FileNotFoundError: [Errno 2] File b'data_seloger_EDAforSpatial_part3.csv' does not exist: b'data_seloger_EDAforSpatial_part3.csv'

After loading the dataset creating in the previous part we need to clean our data as we want to be able to see on the map for each neighborhood:
- The total number of appartments listed
- The lowest rent
- The highest rent
- The average rent
- The median rent
- The average area in square meters
- The median area in square meters
- The average rent per square meters
- The median rent per square meters


A rent per square meters feature is added to neighborhood_data and the dataframe is summarized using groupby and aggregate functions to create the final nbhd_data dataframe with all numeric fields converted to integer values for ease in displaying the data (except for Avg_Rent_SqM and Median_Rent_SqM, we will round them to the first decimal):

In [4]:
# Create a rent_SqM feature
neighborhood_data['Rent_SqM'] = neighborhood_data['rent'] / neighborhood_data['area']

# Create a new dataframe with the new features of interest
#Max_Rent = round(neighborhood_data.groupby('nbhd_no').rent.max(),0)
nbhd_data = neighborhood_data.groupby(['sector_no', 'sector_name', 'nbhd_no', 'nbhd_name']).agg(Tot_Apt_ForRent=('nbhd_no', 'size'),
                                              Min_Rent=('rent', 'min'), 
                                              Max_Rent=('rent', 'max'), 
                                              Avg_Rent=('rent', np.mean), 
                                              Median_Rent=('rent', np.median), 
                                              Avg_Area=('area', np.mean), 
                                              Median_Area=('area', np.median), 
                                              Avg_Rent_SqM=('Rent_SqM', np.mean), 
                                              Median_Rent_SqM=('Rent_SqM', np.median))

# Convert index of a pandas dataframe into a column
nbhd_data.reset_index('nbhd_name', inplace=True)
nbhd_data.reset_index('nbhd_no', inplace=True)
nbhd_data.reset_index('sector_name', inplace=True)
nbhd_data.reset_index('sector_no', inplace=True)

# Convert to integer
cols_round0 = ['Tot_Apt_ForRent', 
               'Min_Rent', 'Max_Rent', 'Avg_Rent','Median_Rent', 
               'Avg_Area', 'Median_Area']
for i in cols_round0:
    nbhd_data = nbhd_data.astype({i: 'int'})
    
# Round to the first decimal 
cols_round1 = ['Avg_Rent_SqM','Median_Rent_SqM']
nbhd_data[cols_round1] = nbhd_data[cols_round1].round(1)

nbhd_data.sort_values(by=['nbhd_no'])

,sector_no,sector_name,nbhd_no,nbhd_name,Tot_Apt_ForRent,Min_Rent,Max_Rent,Avg_Rent,Median_Rent,Avg_Area,Median_Area,Avg_Rent_SqM,Median_Rent_SqM
0,sector1,TOULOUSE CENTRE,n1_1,Capitole - Arnaud Bernard - Carmes,475,280,2917,787,676,52,44,16.7,15.6
1,sector1,TOULOUSE CENTRE,n1_2,Amidonniers - Compans Caffarelli,48,428,1750,749,644,48,41,17.8,16.9
2,sector1,TOULOUSE CENTRE,n1_3,Les Chalets/Bayard/Belfort Saint-Aubin/Dupuy,35,377,1850,771,698,51,40,17.8,16.4
3,sector2,TOULOUSE RIVE GAUCHE,n2_1,Saint-Cyprien,58,377,2000,729,702,50,49,16.1,14.5
4,sector2,TOULOUSE RIVE GAUCHE,n2_2,Croix de Pierre - Route d'Espagne,11,451,990,690,697,56,55,13.7,11.9
5,sector2,TOULOUSE RIVE GAUCHE,n2_3,Fontaine-Lestang - Arènes -Bagatelle - Papus -...,26,430,973,677,646,58,60,12.0,11.7
6,sector2,TOULOUSE RIVE GAUCHE,n2_4,Casselardit - Fontaine-Bayonne - Cartoucherie,13,530,930,706,720,55,56,12.9,13.0
7,sector3,TOULOUSE NORD,n3_1,Minimes - Barrière de Paris - Ponts-Jumeaux,114,352,1473,671,668,53,54,13.2,12.3
8,sector3,TOULOUSE NORD,n3_2,Sept Deniers - Ginestous - Lalande,176,409,1235,640,640,55,58,11.9,11.6
9,sector3,TOULOUSE NORD,n3_3,Trois Cocus - Borderouge - Croix Daurade - Pal...,213,386,1095,660,667,57,60,12.0,11.4


We now need to map this data onto a Toulouse neighborhood map.

## Prepare the Mapping Data and GeoDataFrame
"We will be working with GeoJSON, a popular open standard for representing geographical features with JSON. JSON (JavaScript Object Notation), is a minimal, readable format for structuring data. Bokeh uses JSON to transmit data between a bokeh server and a web application.

In a typical Bokeh interactive graph the data source needs to be a ColumnDataSource. This is a key concept in Bokeh. However, when using a map we use a GeoJSONDataSource instead.

To make our work with geospatial data in Python easier we use GeoPandas. It combines the capabilities of pandas and shapely, providing geospatial operations in pandas and a high-level interface to multiple geometries to shapely. We will use GeoPandas to create a GeoDataFrame - a precursor to creating the GeoJSONDataSource." Jim King

Finally, we need a map that is in GeoJSON format. Toulouse, through their website https://data.toulouse-metropole.fr/, has some exportable neighborhood maps in GeoJSON format providing various demographic. We will import one of them into a GeoDataframe object.

In [5]:
# Read the geojson map file for Realtor Neighborhoods into a GeoDataframe object
tlse = geopandas.read_file('recensement-population-2015-grands-quartiers-population.geojson')
tlse.head()

,p15_f6074,c15_f15p_cs5,p15_pop_etr,c15_pop15p_cs4,p15_pop4559,libelle_des_grands_quartiers,p15_pop0014,c15_f15p_cs3,p15_f1529,c15_f15p_cs1,p15_h3044,p15_popf,c15_f15p_cs6,code_insee,c15_f15p_cs4,p15_pop80p,p15_poph,c15_f15p_cs8,c15_h15p,p15_h2064,c15_h15p_cs8,p15_pop3044,p15_h0014,p15_pop_imm,p15_pop,reg2016,p15_h0019,p15_pop2539,p15_f4559,p15_f3044,c15_pop15p_cs3,p15_pop6074,p15_pop65p,p15_f75p,p15_pop6579,p15_phormen,p15_h65p,p15_pop5564,p15_pop0002,c15_h15p_cs5,c15_h15p_cs4,c15_h15p_cs7,c15_h15p_cs6,p15_pop75p,p15_h4559,c15_h15p_cs3,c15_h15p_cs2,p15_f2064,uu2010,p15_f0014,p15_pop1824,p15_pop4054,c15_pop15p_cs8,p15_h75p,p15_pop1529,p15_pop0305,p15_f0019,p15_pop0610,p15_pmen,p15_pop_fr,c15_f15p,c15_f15p_cs2,dep,c15_pop15p_cs1,c15_pop15p_cs2,c15_pop15p,c15_h15p_cs1,c15_pop15p_cs5,c15_pop15p_cs6,c15_pop15p_cs7,p15_h6074,p15_h1529,grd_quart,p15_f65p,p15_pop0019,c15_f15p_cs7,p15_pop1117,p15_pop2064,geometry
0,447.023010,733.460263,866.535443,1315.200776,1206.140960,SAINT-AUBIN - DUPUY,714.861379,799.890603,2154.702908,0.000000,1017.641412,310.607230,107.346705,31555,758.364483,269.345699,4405.938205,1265.933655,3994.676704,3458.972088,970.193928,1876.969492,414.261503,1033.669606,8983.998180,76,636.358886,2945.186334,579.079154,859.328080,1965.555969,779.700237,846.622167,237.326948,577.276468,76.000039,4578.059975,637.088480,173.196937,416.435772,556.836293,328.006434,324.492897,384.609451,627.061806,1165.665366,225.623314,3321.651766,31701,300.599876,2394.854618,1301.964558,2236.127583,147.282503,4021.716662,138.268310,720.393273,214.200688,8907.998141,8117.462737,4283.460102,71.362556,31,7.422699,296.985871,8278.136806,7.422699,1149.896034,431.839601,875.108272,332.677227,1867.013754,3155511,536.014936,1356.752159,547.101838,332.616090,6780.623854,"POLYGON ((1.45183 43.60241, 1.45158 43.60284, ..."
1,471.600001,514.019566,1021.146864,948.548704,1007.729444,MATABIAU,535.041625,586.868126,1500.964264,3.761838,839.289910,492.511919,44.324396,31555,518.812649,377.379047,3825.654283,1043.959751,3549.408588,2838.405351,856.208790,1432.109779,274.597883,1136.725393,7502.563625,76,494.737014,2092.904866,501.948530,592.819869,1413.531263,900.951698,1136.579617,349.132935,759.200570,53.000027,3676.909342,640.587678,161.669251,435.934427,429.736054,477.756570,370.081224,588.813447,505.780915,826.663137,143.791962,2501.542688,31701,260.443742,1894.633243,1100.676714,1900.168541,239.680512,3037.917631,103.771195,531.298955,117.332001,7449.563598,6481.416761,3415.465599,60.485695,31,12.998262,204.277657,6964.874187,9.236424,949.953994,414.405619,1120.990148,429.351697,1536.953367,3155510,644.067698,1026.035969,643.233578,254.409059,5339.948039,"POLYGON ((1.44541 43.61070, 1.44675 43.61154, ..."
2,39.907418,29.210149,373.524947,36.180255,95.724497,GINESTOUS,348.972271,16.979351,138.437169,0.000000,89.320579,58.999975,3.885828,31555,18.136291,105.184927,468.544155,269.165526,287.853363,228.442237,88.687510,224.522036,172.101938,381.427260,1129.425368,76,181.101943,243.245632,61.808529,135.201457,21.074426,71.986268,203.411714,108.656307,98.226787,424.524027,660.881212,44.802530,48.021203,40.113072,18.043964,55.601600,19.225678,139.672507,33.915968,4.095075,62.086464,283.370964,31701,176.870333,146.382949,111.423351,357.853036,31.016200,248.547789,79.703485,233.098509,104.915867,704.901341,755.900421,486.628767,3.955839,31,0.000000,66.042304,774.482130,0.000000,69.323221,23.111506,200.897382,32.078850,110.110621,3155536,144.411739,414.200452,145.295783,147.518636,511.813201,"POLYGON ((1.42352 43.64833, 1.42323 43.64673, ..."
3,281.426329,1562.004215,973.397567,2183.787900,1290.066448,LALANDE,1914.352190,353.033214,1891.855641,0.000000,1548.314173,333.917576,185.276845,31555,1128.552558,334.270652,5305.954959,630.103100,4368.570065,3812.628230,361.024024,2945.342514,948.116243,1370.247044,10717.875158,76,1159.409153,4294.801539,618.384960,1397.028341,1023.596031,567.611210,728.652473,256.988981,394.381820,172.666755,5411.920198,633.830202,608.

A key column of the data is the neighborhood code (grd_quart) which needs to match the mapping code for the neighborhood. This will allow us to merge the data with the map. Before merging the data we then need to make sure the neighborhoods in both files do match.

First let's take a look at the neighborhoods (column 'libelle_des_grands_quartiers') displayed in nbhd_data Dataframe:

In [6]:
print('Numbers of unique neighborhoods in nbhd_data: {} '.format(nbhd_data['nbhd_name'].describe()))

Numbers of unique neighborhoods in nbhd_data: count                                    20
unique                                   20
top       Croix de Pierre - Route d'Espagne
freq                                      1
Name: nbhd_name, dtype: object 


There are 20 unique neighborhoods in nbhd_data Dataframe. 
Now let's take a look at the neighborhoods (column 'libelle_des_grands_quartiers') displayed in tlse GeoDataframe:

In [7]:
print('Numbers of unique neighborhoods in tlse: {} '.format(tlse['libelle_des_grands_quartiers'].describe()))

Numbers of unique neighborhoods in tlse: count          60
unique         60
top       EMPALOT
freq            1
Name: libelle_des_grands_quartiers, dtype: object 


There are 60 unique neighborhoods in tlse GeoDataframe and thus three times more neighborhoods in the file imported from the Toulouse website. By taking a visual look at the neighborhood names we identify that each neighborhood have been divided in smaller ones in the GeoDataFrame.  To fix this issue we need to: 
1. Create a dictionary to change the neighborhood codes in the map to match the neighborhood codes in the data
2. Dissolve the polygons Based On an the new neighborhood codes

In [59]:
# Create a dictionary to change the neighborhood codes to neighborhood names.
nbhd_dict = {'3155507': 'n1_2','3155533': 'n2_3','3155502': 'n1_1','3155532': 'n2_3',
             '3155537': 'n3_1','3155556': 'n6_2','3155552': 'n6_3','3155519': 'n4_1',                            
             '3155501': 'n1_1','3155505': 'n1_1','3155535': 'n2_4','3155545': 'n4_3',                            
             '3155508': 'n1_2','3155522': 'n4_3','3155540': 'n3_3','3155528': 'n2_2',
             '3155527': 'n5_3','3155530': 'n2_3','3155515': 'n2_1','3155531': 'n2_3',
             '3155536': 'n3_2','3155541': 'n4_2','3155521': 'n4_3','3155526': 'n5_2',                        
             '3155543': 'n4_2','3155534': 'n6_2','3155551': 'n6_4','3155546': 'n5_1',
             '3155538': 'n3_2','3155558': 'n6_2','3155512': 'n5_3','3155509': 'n1_3',
             '3155539': 'n3_3','3155557': 'n6_2','3155520': 'n4_1','3155510': 'n1_3',
             '3155518': 'n3_1','3155554': 'n6_3','3155547': 'n5_1','3155529': 'n2_3',
             '3155516': 'n2_1','3155523': 'n5_1','3155549': 'n5_3','3155560': 'n6_1',
             '3155514': 'n5_3','3155548': 'n5_2','3155553': 'n6_3','3155542': 'n4_2',
             '3155525': 'n5_3','3155511': 'n1_3','3155506': 'n2_1','3155504': 'n1_1',               
             '3155503': 'n1_1','3155559': 'n6_1','3155513': 'n5_3','3155555': 'n6_4',
             '3155524': 'n5_2','3155517': 'n3_2','3155544': 'n4_2','3155550': 'n6_4'}

# Create a neighborhood name from the dictionary neighborhood_dict
tlse['nbhd_no'] = tlse['grd_quart'].map(nbhd_dict)

In [60]:
# select the columns that you wish to retain in the data
tlse_short = tlse[['nbhd_no', 'geometry']]

# then summarize the quantative columns by 'sum' 
tlse_agg = tlse_short.dissolve(by='nbhd_no', aggfunc = 'sum')

# Convert index of a pandas dataframe into a column
tlse_agg.reset_index('nbhd_no', inplace=True)

tlse_agg.columns

Index(['nbhd_no', 'geometry'], dtype='object')

We use geopandas to read the geojson map into the GeoDataFrame sf. We then set the coordinate reference system to lat-long projection. Next, we rename several columns and use set_geometry to set the GeoDataFrame to column ‘geometry’ containing the active geometry (the description of the shapes to draw). Finally, we clean up some neighborhood id’s to match neighborhood_data.

In [97]:
# Set the Coordinate Referance System (crs) for projections
# ESPG code 4326 is also referred to as WGS84 lat-long projection
tlse_agg.crs = {'init': 'epsg:4326'}

We now have our neighborhood data in nbhd_data and our mapping data in tlse with both sharing the neighborhood code column subdist_no.

## Create the Interactive Plot

#### Create the JSON Data for the GeoJSONDataSource

We now need to merges our neighborhood data with our mapping data and converts it into JSON format for the Bokeh server.

In [62]:
# Merge the GeoDataframe object (tlse_agg) with the neighborhood summary data (neighborhood)
merged = pd.merge(tlse_agg, nbhd_data, on='nbhd_no', how='left')

# Bokeh uses geojson formatting, representing geographical features, with json
# Convert to json
merged_json = json.loads(merged.to_json())

# Convert to json preferred string-like object 
json_data = json.dumps(merged_json)

#### Create The ColorBar
The ColorBar is “attached” to the plot and the entire plot needs to be refreshed when a change in the criteria is requested. Each criteria has it’s own unique minimum and maximum range, format for displaying and verbage. For example, Number of Appartment For Rent has a range of 0–100, a format as an integer and the name 'Number of Appartment For Rent' that needs to be changed in the title of the plot.

So we need to create a format_df that details the data needed in the ColorBar and title.

In [63]:
merged.describe()

,Tot_Apt_ForRent,Min_Rent,Max_Rent,Avg_Rent,Median_Rent,Avg_Area,Median_Area,Avg_Rent_SqM,Median_Rent_SqM
count,20.00000,20.000000,20.000000,20.00000,20.0000,20.000000,20.000000,20.000000,20.000000
mean,93.00000,395.850000,1537.200000,687.50000,657.6000,52.800000,52.550000,14.015000,13.055000
std,104.40609,51.442635,678.565096,52.91055,43.6293,3.750088,7.680974,2.013971,1.764109
min,11.00000,280.000000,871.000000,598.00000,566.0000,44.000000,40.000000,11.500000,11.100000
25%,35.75000,374.500000,985.750000,654.50000,643.0000,50.750000,44.000000,12.525000,11.675000
50%,63.00000,395.000000,1292.500000,677.50000,667.5000,53.500000,55.500000,13.250000,12.400000
75%,103.00000,413.750000,1887.500000,725.25000,691.7500,55.250000,59.250000,15.500000,14.125000
max,475.00000,530.000000,2917.000000,787.00000,720.0000,58.000000,60.000000,17.800000,16.900000


In [126]:
# This dictionary contains the formatting for the data in the plots
format_data = [('Tot_Apt_ForRent', 0, 500, '0,0', 'Number of Appartment For Rent'),
               ('Min_Rent', 250, 550, '$0,0 ', 'Minimum Rent'),
               ('Max_Rent', 850, 3000, '0,0', 'Maximum Rent'),
               ('Avg_Rent', 550, 800, '$0,0', 'Average Rent'),
               ('Median_Rent', 550, 750, '$0,0', 'Median Rent'),
               ('Avg_Area', 40, 60, '0,0', 'Average Area in Square Meters'),
               ('Median_Area', 40, 60, '0,0', 'Median Area in Square Meters'),
               ('Avg_Rent_SqM', 11, 18, '$0,0', 'Average Rent per Square Meter'),
               ('Median_Rent_SqM', 11, 18, '$0,0', 'Median Rent per Square Meter')]
 
#Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

The callback function update_plot has three parameters. The attr parameter is simply the ‘value’ you passed (e.g. slider.value or select.value), the old and new are internal parameters used by Bokeh and you do not need to deal with them.

We select re-set the input_field (Select) based on criteria (cr) before re-seting the plot based on the current input_field.

In [127]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    # The input cr is the criteria selected from the select box
    cr = select.value
    input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]
    
    # Update the plot based on the changed inputs
    p = make_plot(input_field)
    
    # Update the layout, clear the old document and display the new document
    layout = column(p, widgetbox(select))
    curdoc().clear()
    curdoc().add_root(layout)
    
    # Update the data
    geosource.geojson = json_data 

#### Create a Plotting Function
The final piece of the map is make_plot, the plotting function. Let’s break this down:
1. We pass it the field_name to indicate which column of data we want to plot (e.g. Median Sales Price).
2. Using the format_df we pull out the minimum range, maximum range and formatting for the ColorBar.
3. We call Bokeh’s LinearColorMapper to set the palette and range of the colorbar.
4. We create the ColorBar using Bokeh’s NumeralTickFormatter and ColorBar.
5. We create the plot figure with appropriate title.
6. We create the “patches”, in our case the neighborhood polygons, using Bokeh’s p.patches glyph using the data in geosource.
7. We add the colorbar and the HoverTool to the plot and return the plot p.

In [128]:
# Create a plotting function
def make_plot(field_name):    
    # Set the format of the colorbar
    min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
    max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
    field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

    # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

    # Create color bar.
    format_tick = NumeralTickFormatter(format=field_format)
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
    border_line_color=None, location = (0, 0))

    # Create figure object.
    verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

    p = figure(title = verbage + ' by Neighborhood for Appartments for Rent in Toulouse (2020)', 
             plot_height = 650, plot_width = 850,
             toolbar_location = None)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False

    # Add patch renderer to figure. 
    p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

    # Specify color bar layout.
    p.add_layout(color_bar, 'right')

    # Add the hover tool to the graph
    p.add_tools(hover)
    return p

#### Main Code for Interactive Map
We still need several pieces of code to make the interactive map including a ColorBar, Bokeh widgets and tools, a plotting function and an update function, but before we go into detail on those pieces let’s take a look at the main code.

In [194]:
# Input geojson source that contains features for plotting for:
# initial year 2018 and initial criteria sale_price_median
geosource = GeoJSONDataSource(geojson = json_data)

# Define a sequential multi-hue color palette.
palette = brewer['Purples'][9] # "La Garonne est viola"

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#### The HoverTool
The HoverTool is a fairly straightforward Bokeh tool that allows the user to hover over an item and display values. In the main code we insert HoverTool code and tell it to use the data based on the neighborhood_name and display the six criteria using “@” to indicate the column values.

In [191]:
# Add hover tool
hover = HoverTool(tooltips = [ ('Sector','@sector_name'),
                               ('Neighborhood','@nbhd_name'),
                               ('#Apt. For Rent', '@Tot_Apt_ForRent available'),
                               ('Average Rent', '@Avg_Rent{,} €'),
                               ('Median Rent', '@Median_Rent{,} €'),
                               ('Median Area', '@Median_Area{,} SqM'),
                               ('Median Rent/SqM', '@Median_Rent_SqM{0.2f} €/SqM'),
                               ('Minimum Rent', '@Min_Rent{,} €'),
                               ('Maximum Rent', '@Max_Rent{,} €')])
               
# Call the plotting function
input_field = 'Median_Rent_SqM'
p = make_plot(input_field)

#### Widgets and The Callback Function
We need to use a Bokeh widgets, more precisely a Select object allows the user to select the criteria (or column).
This widget works on the following principle - the callback. 
In the code below, the widgets pass a ‘value’ and call a function named update_plot when the .on_change method is called (when a change is made using the widget - the event handler).

In [192]:
# Make a selection object: select
select = Select(title='Select Criteria:', value='Median Sales Price', options=['Median Rent', 'Average Rent',
                                                                               'Median Rent per Square Meter', 'Average Rent per Square Meter',
                                                                               'Median Area in Square Meters', 'Average Area in Square Meters',
                                                                               'Minimum Rent','Maximum Rent',
                                                                               'Number of Appartment For Rent'])

select.on_change('value', update_plot)

#### The Static Map with ColorBar and HoverTool

In [196]:
# Use the following code to test in a notebook, comment out for transfer to live site
# Interactive features will not show in notebook
output_notebook()
show(p)

Loading BokehJS ...

In [197]:
output_file('test.html')
show(p)

#### The Bokeh Server
I developed the static map using 2018 data and Median Sales Price in Colab in order to get the majority of the code working prior to adding the interactive portions. In order to test and view the interactive components of Bokeh, you will need to follow these steps.
1. Install the Bokeh server on your computer.
2. Download the .ipynb file to a local directory on your computer.
3. From the terminal change the directory to the directory with the .ipynb file.
4. From the terminal run the following command: bokeh serve (two dashes)show filename.ipynb
5. This should open a local host on your browser and output your interactive graph. If there is an error it should be visible in the terminal.

Package                            Version    
---------------------------------- -----------
alabaster                          0.7.12     
anaconda-client                    1.7.2      
anaconda-navigator                 1.9.7      
anaconda-project                   0.8.3      
asn1crypto                         1.0.1      
astroid                            2.3.1      
astropy                            3.2.1      
atomicwrites                       1.3.0      
attrs                              19.2.0     
Babel                              2.7.0      
backcall                           0.1.0      
backports.functools-lru-cache      1.5        
backports.os                       0.1.1      
backports.shutil-get-terminal-size 1.0.0      
backports.tempfile                 1.0        
backports.weakref                  1.0.post1  
beautifulsoup4                     4.8.0      
bitarray                           1.0.1      
bkcharts                           0.2        
bleach       

In [170]:
bokeh serve:show filename.ipynb

SyntaxError: invalid syntax (<ipython-input-170-128d04eb808b>, line 1)

In [ ]:
import Jinja2
import packaging
import pillow
import dateutil
import PyYAML
import six
import tornado
import Futures
import bokeh

#### Public Access to the Interactive Graph via Heroku
Once you get the interactive graph working locally, you can let others access it by using a public Bokeh hosting service such as Heroku. Heroku will host the interactive graph allowing you to link to it (as in this article) or use an iframe such as on my GitHub Pages site.
The basic steps to host on Heroku are:
1. Change the Colab notebook to comment out the install of fiona and geopandas. Heroku has these items and the build will fail if they are in the code.
2. Change the Colab notebook to comment out the last two lines (output_notebook() and show(p)).
3. Download the Colab notebook as a .py file and upload it to a GitHub repository.
4. Create a Heroku app and connect to your GitHub repository containing your .py file.
5. Create a Procfile and requirements.txt file. See mine in my GitHub.
6. Run the app!
